In [1]:
import numpy as np
import mxnet as mx
from bqplot import *
from bqplot.marks import Graph
from ipywidgets import IntSlider, Dropdown, RadioButtons, HBox, VBox, Button, Layout
from bqplot import pyplot as plt
from bqplot import OrdinalScale
import itertools
import pdb
import warnings
warnings.simplefilter('ignore')
from IPython.display import display

/Users/maded/dev/bqplot/bqplot/scales.py:433: DeprecationWarning: metadata {'args': (None,)} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  min = Date(default_value=None, allow_none=True).tag(sync=True)
/Users/maded/dev/bqplot/bqplot/marks.py:827: DeprecationWarning: metadata {'sync': True, 'display_name': 'Fill color for the box'} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  box_fill_color = Color('dodgerblue', sync=True, display_name='Fill color for the box')
/Users/maded/dev/bqplot/bqplot/marks.py:993: DeprecationWarning: metadata {'display_name': 'Marker'} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  marker = Enum(['candle', 'bar'], default_value='candle', display_name='Marker').tag(syn

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
from mxnet import gluon
from mxnet.gluon import nn, Block, autograd
from sklearn.preprocessing import StandardScaler

In [4]:
num_epochs = 10

In [5]:
X_train = pd.read_csv('data_files/cleaned_train.csv')
X_test = pd.read_csv('data_files/cleaned_test.csv')
y_train = X_train['label']
y_test = X_test['label']
X_train['RevolvingUtilizationOfUnsecuredLinesind'] = X_train['RevolvingUtilizationOfUnsecuredLinesind'].astype('float')
X_train['overdue_ind'] = X_train['overdue_ind'].astype('float')

X_train = X_train.drop('label', axis=1)
X_test = X_test.drop('label', axis=1)

scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

X_train = mx.nd.array(X_train)
y_train = mx.nd.reshape(mx.nd.array(y_train), (y_train.shape[0], 1))

X_test = mx.nd.array(X_test)
y_test = mx.nd.reshape(mx.nd.array(y_test), (y_test.shape[0], 1))

In [6]:
class MLP(gluon.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(13)
            self.dense1 = nn.Dense(5)
            self.dense2 = nn.Dense(1)

    def forward(self, x):
        x1 = mx.nd.relu(self.dense0(x))
        x2 = mx.nd.relu(self.dense1(x1))
        x3 = self.dense2(x2)
        return x3, x2, x1
    
net = MLP()

In [7]:
bce_loss = gluon.loss.SigmoidBinaryCrossEntropyLoss()

In [8]:
auc_fig = plt.figure(title='Train and Test AUC Score', legend_location='top-left')
auc_line = plt.plot([0], [0], marker='circle', marker_size=32, colors=['DeepSkyBlue', 'Red'], 
                              labels=['Training', 'Test'], display_legend=True)
plt.xlabel('Epoch')

train_accs = []
val_accs = []

# data

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(X_train, y_train),
    batch_size=100, shuffle=True)

val_data = gluon.data.DataLoader(gluon.data.ArrayDataset(X_test, y_test),
    batch_size=100, shuffle=False)

In [9]:
def test(ctx, train=False):
    outputs = []
    labels = []
    if train is True:
        iter_data = train_data
    else:
        iter_data = val_data
    for data, label in iter_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        labels += label.asnumpy().flatten().tolist()
        output = net(data)[0]
        #pdb.set_trace()
        predictions = output
        outputs += predictions.asnumpy().flatten().tolist()
    
    return 'accuracy', roc_auc_score(np.array(labels), np.array(outputs))


def train(epochs, ctx):
    # Collect all parameters from net and its children, then initialize them.
    net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
    # Trainer is for updating parameters with gradient.
    trainer = gluon.Trainer(net.collect_params(), 'adam',
                            {'learning_rate': 0.01})

    for epoch in range(epochs):
        for i, (data, label) in enumerate(train_data):
            # Copy data to ctx if necessary
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            with autograd.record():
                outputs = net(data)
                L = bce_loss(outputs[0], label)
                L.backward()
            trainer.step(data.shape[0])

        name, train_acc = test(ctx, train=True)
        train_accs.append(train_acc)
        auc_line.x = np.arange(1, epoch+1)

        name, val_acc = test(ctx)
        val_accs.append(val_acc)
        auc_line.y = [train_accs, val_accs]

        net.save_params('data/epoch-{}.params'.format(epoch + 1))

display(auc_fig)
ctx = mx.cpu()
train(10, ctx)

Figure(axes=[Axis(label='Epoch', scale=LinearScale(), side='bottom'), Axis(orientation='vertical', scale=LinearScale(), side='left')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), legend_location='top-left', marks=[Lines(colors=['DeepSkyBlue', 'Red'], display_legend=True, interactions={'hover': 'tooltip'}, labels=['Training', 'Test'], marker='circle', marker_size=32, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([0]), y=array([0]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Train and Test AUC Score')

In [10]:
from gluon_mlp_dashboard import MLPDashboard

In [11]:
data = list(itertools.islice(train_data, 1))[0][0]

In [12]:
a = MLPDashboard(net, 'nn-params', 'epoch', data=data)
a

MLPDashboard(children=(HBox(children=(IntSlider(value=1, description='Epoch:', max=10, min=1), Dropdown(description='View', options=('Weights', 'Gradients', 'Activations'), value='Weights'), Button(description='Update', style=ButtonStyle()))), Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='800px', min_width='125px', width='900px'), marks=[Graph(colors=['Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange'], directed=False, hovered_style={'stroke': '1.5'}, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 13, 'value': 0}, {'source': 0, 'target': 14, 'value': 0}, {'source': 0, 'target': 15, 'value': 0}, {'source': 0, 'target': 16, 'value': 0}, {'source': 0, 'target': 17, 'value': 0}, {'source': 0, 'target': 18, 'value': 0}, {'source': 0, 'target': 19, 'value': 0}, {'source': 0, 'target': 20, 'value': 0}, {'source': 0, 'target': 21, 'value': 0}, {'source': 0, 'target': 22, 'value': 0}, {'source': 0, 'target': 23, 'value': 0}, {'source': 0, 'target': 24, 'value': 0}, {'source': 0, 'target': 25, 'value': 0}, {'source': 1, 'target': 13, 'value': 0}, {'source': 1, 'target': 14, 'value': 0}, {'source': 1, 'target': 15, 'value': 0}, {'source': 1, 'target': 16, 'value': 0}, {'source': 1, 'target': 17, 'value': 0}, {'source': 1, 'target': 18, 'value': 0}, {'source': 1, 'target': 19, 'value': 0}, {'source': 1, 'target': 20, 'value': 0}, {'source': 1, 'target': 21, 'value': 0}, {'source': 1, 'target': 22, 'value': 0}, {'source': 1, 'target': 23, 'value': 0}, {'source': 1, 'target': 24, 'value': 0}, {'source': 1, 'target': 25, 'value': 0}, {'source': 2, 'target': 13, 'value': 0}, {'source': 2, 'target': 14, 'value': 0}, {'source': 2, 'target': 15, 'value': 0}, {'source': 2, 'target': 16, 'value': 0}, {'source': 2, 'target': 17, 'value': 0}, {'source': 2, 'target': 18, 'value': 0}, {'source': 2, 'target': 19, 'value': 0}, {'source': 2, 'target': 20, 'value': 0}, {'source': 2, 'target': 21, 'value': 0}, {'source': 2, 'target': 22, 'value': 0}, {'source': 2, 'target': 23, 'value': 0}, {'source': 2, 'target': 24, 'value': 0}, {'source': 2, 'target': 25, 'value': 0}, {'source': 3, 'target': 13, 'value': 0}, {'source': 3, 'target': 14, 'value': 0}, {'source': 3, 'target': 15, 'value': 0}, {'source': 3, 'target': 16, 'value': 0}, {'source': 3, 'target': 17, 'value': 0}, {'source': 3, 'target': 18, 'value': 0}, {'source': 3, 'target': 19, 'value': 0}, {'source': 3, 'target': 20, 'value': 0}, {'source': 3, 'target': 21, 'value': 0}, {'source': 3, 'target': 22, 'value': 0}, {'source': 3, 'target': 23, 'value': 0}, {'source': 3, 'target': 24, 'value': 0}, {'source': 3, 'target': 25, 'value': 0}, {'source': 4, 'target': 13, 'value': 0}, {'source': 4, 'target': 14, 'value': 0}, {'source': 4, 'target': 15, 'value': 0}, {'source': 4, 'target': 16, 'value': 0}, {'source': 4, 'target': 17, 'value': 0}, {'source': 4, 'target': 18, 'value': 0}, {'source': 4, 'target': 19, 'value': 0}, {'source': 4, 'target': 20, 'value': 0}, {'source': 4, 'target': 21, 'value': 0}, {'source': 4, 'target': 22, 'value': 0}, {'source': 4, 'target': 23, 'value': 0}, {'source': 4, 'target': 24, 'value': 0}, {'source': 4, 'target': 25, 'value': 0}, {'source': 5, 'target': 13, 'value': 0}, {'source': 5, 'target': 14, 'value': 0}, {'source': 5, 'target': 15, 'value': 0}, {'source': 5, 'target': 16, 'value': 0}, {'source': 5, 'target': 17, 'value': 0}, {'source': 5, 'target': 18, 'value': 0}, {'source': 5, 'target': 19, 'value': 0}, {'source': 5, 'target': 20, 'value': 0}, {'source': 5, 'target': 21, 'value': 0}, {'source': 5, 'target': 22, 'value': 0}, {'source': 5, 'target': 23, 'value': 0}, {'source': 5, 'target': 24, 'value': 0}, {'source